In [1]:
import sys
import os.path
from subprocess import check_call
import importlib
import os

modules = ["pymongo","pprint"]
print("Checking required modules")
for m in modules:
    torch_loader = importlib.util.find_spec(m)
    if torch_loader is not None:
        print(m," found")
    else:
        print(m," not found, installing")
        if 'google.colab' in sys.modules:
            check_call(["pip", "install", "-q", m])
        else:
            check_call([sys.executable, "-m", "pip", "install", "--user", m])
print("Done!")

Checking required modules
pymongo  found
pprint  found
Done!


In [2]:
from pprint import pprint # para mostrar los json bonitos
from pymongo import MongoClient

# conexión a mongo
servidor = 'localhost'
puerto = 27017

basededatos = 'mirko'

client = MongoClient(servidor, puerto)
db = client[basededatos]

In [3]:
import json # para transformar la línea leida (string) a json
import urllib.request # para leer de la URL línea a línea


def carga_desde_fichero(db,fichero,coleccion):
    db[coleccion].drop() # la borramos
    exito,error = 0,0
    # cargamos los datos desde el fichero
    try:
        with urllib.request.urlopen(fichero) as f:
        
            for line in f:
                line2 = line.decode("UTF-8").replace("$","")
                res = db[coleccion].insert_one(json.loads(line2))
                if res.acknowledged:
                    exito+=1
                else:
                    error+=1
        print(f"Colección {coleccion}: {exito} documentos cargados con éxito y {error} errores")
    except urllib.error.URLError as e:
        print(e.reason)
    return
   


fichero_user = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/userrentaraw.json"
carga_desde_fichero(db,fichero_user,"user")


Colección user: 1697 documentos cargados con éxito y 0 errores


Ejemplo:

renta: "Almería, Almería\nDistrito: 07 Sección: 016\nRenta mediana 201812.950 €Cambio 2015-18+12 %\nLa renta de esta sección está entre el 18 y el 25% más pobre de España y el 36 y el 45% más rico de su comunidad"

convertir en:

"datosrenta" : {"Localidad": "Almería"
"Provincia": "Almería"
"Distrito": "07"
"Sección": "016"
"RentaMediana": 201812.9,
"cambio1518": +12
"rentapais":[18,25]
"rentacom":[55,64]
}




In [4]:
import re
def quita_barra(dir):
    if "/" in dir:
        #print("Quitando barra de ",dir)
        pos = dir.index("/")
        #print("/ pos",pos)
        if "," in dir[pos:]:
            poscoma = dir[pos:].index(",")
            #print("pos coma",poscoma)
            dir = dir[:pos]+dir[(pos+poscoma):]
            #print("dir cambiado",dir)
            dir = quita_barra(dir) # por si hay más barras
    return dir

cursor = db.user.find()
for doc in cursor:
    renta = quita_barra(doc["renta"])
    
    #provincias pais vasco y navarra
    provinciasQuitar = ["Gipuzcoa","Bizkaia","Álava","Navarra"]
    
    datosrenta =  {}
    #print(renta)
    if renta=="":
        db.user.delete_one({"_id":doc["_id"]})
    # tambien hay que borrar los del pais vasco
    
    elif any([no in renta for no in provinciasQuitar]):
        db.user.delete_one({"_id":doc["_id"]})
    else:
        # localidad
        posDistrito = renta.index("Distrito")
        hastaDistrito = renta[0:(posDistrito-1)]
        posUltComa = hastaDistrito.rindex(",")
        localidad = hastaDistrito[:posUltComa]
        datosrenta["localidad"]=localidad.strip()
        
        # provincia
        datosrenta["provincia"]= hastaDistrito[posUltComa+1:].strip()
        
        # distrito
        trasDistrito = renta[posDistrito:]
        posdospuntos = trasDistrito.index(":")
        posSeccion = trasDistrito.index("Sección:")
        datosrenta["distrito"] = trasDistrito[(posdospuntos+1):posSeccion].strip()
        
        # sección
        trasSeccion = trasDistrito[posSeccion:]
        posdospuntos2 = trasSeccion.index(":")
        posRentaMed = trasSeccion.index("Renta mediana 2018")
        datosrenta["sección"] = trasSeccion[(posdospuntos2+1):posRentaMed].strip()
        
        # renta mediana
        posIni = posRentaMed + len("Renta mediana 2018")
        posEuro = trasSeccion.index("€")
        datosrenta["renta mediana"] = trasSeccion[posIni:posEuro].strip()
        
        # cambio1518
        posCambio1518 = renta.index("Cambio 2015-18")
        posIni=posCambio1518+len("Cambio 2015-18")
        trasCambio=renta[posIni:]
        posSiguiente=trasCambio.index("%")
        datosrenta["cambio1518"] = trasCambio[:posSiguiente-1].strip()

        # No todas las rentas presentan información de riqueza y pobreza
        infoPobre= trasCambio[posSiguiente+1:]
        
        # Pobre España        
        if (len(infoPobre)):
            # Separacion (cualquiera) entre números de España y comunidad
            posSeparacion=infoPobre.find("%")
            separacion=infoPobre[:posSeparacion] 
            listaPais=[]
            # Encuentro los enteros correspondientes a España y los añado a una lista
            mapa=(map(int, re.findall(r'\d+', separacion)))
            nex=next (mapa,-1)
            while(nex!=-1):
                listaPais.append(nex)
                nex=next(mapa,-1)
            #listaPais: lista con el porcentaje o intervalo de porcentajes de pobreza respecto al país 

            #Observo si el porcentaje es de pobreza o de riqueza 
            pobreORico=(infoPobre[(posSeparacion+6):])[:5].strip()
            #print(pobreORico,'|',listaPais)

            if(pobreORico=="pobre"):
                datosrenta["rentapais"]=listaPais
            elif(pobreORico=="rico"):
                listaPais.reverse()     
                datosrenta["rentapais"]=[100-i for i in listaPais]
            

            
            # Pobre comunidad  
            # Texto siguiente a la separación
            luego=infoPobre[posSeparacion+1:]
            posPobreCom=luego.find("%")

            # Encuentro los enteros correspondientes a la comunidad y los añado a una lista
            mapa=(map(int, re.findall(r'\d+', luego)))
            listaCom=[]
            nex=next (mapa,-1)
            while(nex!=-1):
                listaCom.append(nex)
                nex=next(mapa,-1)
            #listaCom: lista con el porcentaje o intervalo de porcentajes de pobreza respecto a la comunidad 

            #Observo si el porcentaje es de pobreza o de riqueza 
            pobreORico=(luego[(posPobreCom+6):])[:5].strip()
            #print(pobreORico,'|',listaCom)
            if(pobreORico=="pobre"):
                datosrenta["rentacom"]=listaCom
            elif(pobreORico=="rico"):
                listaCom.reverse()
                datosrenta["rentacom"]=[100-i for i in listaCom]            

            #print(datosrenta["rentapais"],'|',datosrenta["rentacom"])
        
        #grabamos
        datosrenta["rentaraw"] = renta 
        db.user.update_one({"_id":doc["_id"]}, {"$set":{"datosrenta":datosrenta}})

In [6]:
!mongoexport --db=mirko --collection=user  --out=user.json

2021-11-13T22:08:10.076+0100	connected to: mongodb://localhost/
2021-11-13T22:08:10.252+0100	exported 1625 records


para borrar lo que hemos hecho

db.user.update_many({},{"$unset":{"datosrenta":1}})